Hier werden die Portfolio-Methoden angewandt und mithilfe der Performance-Metriken bewertet. Die Ergebnisse werden in CSV-Dateien festgehalten. Das Akquirieren, modifizieren und beschreiben der Daten befindet sich der Übersicht halber im Notebook Datenbasis. Die Daten werden dort als CSV exportiert und hier als Dataframe wieder eingelesen.

Die Korrektheit der implementierten Methoden und Metriken wird in dem separaten Notebook Verifizierung anhand eines kleinen Teilbestandes aufgezeigt, siehe dazu auch beigefügte Excel-Dateien, in denen die Portfolio-Gewichte "per Hand" nachgerechnet und mit den Ergebnissen aus der Python-Implementierung abgeglichen werden.

ACHTUNG: LANGE LAUFZEIT!

## Importe

In [1]:
#Import der notwendigen Bibliotheken, Installation mit pip bereits erfolgt

import yfinance as yf
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize
import math
import random
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram
import riskfolio as rp
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import squareform
from riskfolio import HCPortfolio as hc
from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

## Einlesen der Daten

In [2]:
#Bearbeitete JSE-Daten (Gesamtbestand)

df_JSE = pd.read_csv(
    'df_JSE_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [3]:
df_JSE.info

<bound method DataFrame.info of             4SI.JO   ABG.JO  ACL.JO  ACT.JO       ADH.JO  AEL.JO  \
Date                                                               
2005-06-01     NaN   8240.0  4980.0   130.0   133.701508     NaN   
2005-06-02     NaN   8250.0  4949.0   130.0   131.749695     NaN   
2005-06-03     NaN   8261.0  4949.0   130.0   131.749695     NaN   
2005-06-06     NaN   8290.0  5023.0   130.0   132.725601     NaN   
2005-06-07     NaN   8345.0  4845.0   149.0   131.749695     NaN   
...            ...      ...     ...     ...          ...     ...   
2025-05-26    70.0  16845.0   109.0   160.0  3199.000000  2350.0   
2025-05-27    70.0  16848.0   105.0   160.0  3210.000000  2350.0   
2025-05-28    73.0  17300.0   101.0   174.0  3283.000000  2355.0   
2025-05-29    70.0  17579.0   106.0   166.0  3247.000000  2362.0   
2025-05-30    70.0  17332.0   116.0   160.0  3215.000000  2358.0   

                  AFE.JO  AFH.JO  AFT.JO        AGL.JO  ...  VKE.JO   VOD.JO  \
Dat

In [4]:
#Bearbeitete SP500-Daten (Gesamtbestand)

df_SP500 = pd.read_csv(
    'df_SP500_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [5]:
df_SP500.info

<bound method DataFrame.info of                      A  AAL  AAP        AAPL        ABBV        ABNB  \
Date                                                                   
2005-06-01   17.296137  NaN  NaN    1.439286         NaN         NaN   
2005-06-02   17.303289  NaN  NaN    1.430000         NaN         NaN   
2005-06-03   17.210300  NaN  NaN    1.365714         NaN         NaN   
2005-06-06   17.539343  NaN  NaN    1.354286         NaN         NaN   
2005-06-07   17.324751  NaN  NaN    1.305000         NaN         NaN   
...                ...  ...  ...         ...         ...         ...   
2025-05-23  108.529999  NaN  NaN  195.270004  183.259995  126.720001   
2025-05-27  111.260002  NaN  NaN  200.210007  185.720001  129.399994   
2025-05-28  110.879997  NaN  NaN  200.419998  183.089996  128.669998   
2025-05-29  113.279999  NaN  NaN  199.949997  185.619995  128.360001   
2025-05-30  111.919998  NaN  NaN  200.850006  186.110001  129.000000   

                   ABT       AC

In [6]:
#Reduzierter SP500-Bestand ohne ausgeschiedene Unternehmen und auf Größe des JSE runtergebrochen

df_SP500_reduziert = pd.read_csv(
    'sp500_gefiltert_neuneu.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    dayfirst=True
)

In [7]:
df_SP500_reduziert.info

<bound method DataFrame.info of                      A        AAPL         ABT         ACN         ADI  \
Date                                                                     
2005-06-01   17.296137    1.439286         NaN         NaN         NaN   
2005-06-02   17.303289    1.430000         NaN         NaN         NaN   
2005-06-03   17.210300    1.365714         NaN         NaN         NaN   
2005-06-06   17.539343    1.354286         NaN         NaN         NaN   
2005-06-07   17.324751    1.305000         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2025-05-23  108.529999  195.270004  131.300003  309.579987  210.470001   
2025-05-27  111.260002  200.210007  132.940002  315.429993  216.889999   
2025-05-28  110.879997  200.419998  132.020004  315.989990  215.619995   
2025-05-29  113.279999  199.949997  132.850006  317.730011  215.750000   
2025-05-30  111.919998  200.850006  133.580002  316.820007  213.979996   

     

## Definition der Methoden und Metriken

#### Equally-Weighted Portfolio

In [8]:
#Rückgabe gleicher Gewichte anhand der Anzahl der Unternehmen (Input Returns)

def equally_weighted(returns):
    anzahl_assets = returns.shape[1]

    w0 = 1/anzahl_assets
    gewichte = np.full(anzahl_assets, w0)

    return gewichte

#### Minimum-Variance Portfolio

In [9]:
#Minimum-Variance Portfolio
#Code adaptiert von https://medium.com/@BorisGerat/mean-variance-and-minimum-variance-portfolio-models-in-python-64a5c6b57b2d

def min_variance_opt(matrix):
    
    #Compute the minimum-variance portfolio weights subject to the constraints of:
    #- No short-selling (weights must be >= 0)
    #- Full investment (sum of weights must equal 1)
    
    number_of_assets = matrix.shape[0]

    # Define the objective function to minimize portfolio variance
    def objective(w):
        portfolio_variance = np.dot(w, matrix @ w)
        return portfolio_variance

    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(number_of_assets)]

    # Initial guess: Equal weights for all assets
    w0 = np.ones(number_of_assets) / number_of_assets

    # Solve the optimization problem using Sequential Least Squares Quadratic Programming (SLSQP)
    res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return res.x  # Return the optimal weights

#### Maximum Diversification Portfolio

In [10]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def calc_max_diversification_ratio(weights, matrix):
    
    # gewichtete Volatilität
    average_vol = np.dot(np.sqrt(np.diag(matrix)), weights.T)
    
    # portfolio volatilität
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_vol = np.sqrt(portfolio_variance)
    
    #Bestimmung Ratio
    diversification_ratio = average_vol/portfolio_vol
    
    # Negativer Wert für Minimierung (Maximizieren = Minimieren -)
    return -diversification_ratio

In [11]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def max_diversification_portfolio(matrix):
    
    anzahl_assets = matrix.shape[0]
    
    w0 = 1/anzahl_assets
    w = np.full(anzahl_assets, w0)
    
    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(anzahl_assets)]
    
    res = minimize(calc_max_diversification_ratio, w, bounds=bounds, args=matrix, method='SLSQP', constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    return res.x

#### Inverse Volatility Portfolio

In [12]:
#Code adaptiert von https://www.kaggle.com/code/vijipai/lesson-4-traditional-portfolio-construction-method

def inverse_volatility_portfolio (matrix):
    
    standardabweichungen = np.sqrt(np.diagonal(matrix))
    invertierte_std = 1/standardabweichungen
    summe_invertierte_std = np.sum(invertierte_std)
    
    inverse_volatility_gewichte = invertierte_std / summe_invertierte_std
    
    return inverse_volatility_gewichte

#### Equal Risk Contribution Portfolio

In [13]:
#Code adaptiert von https://medium.com/@deepml1818/building-a-python-based-risk-parity-portfolio-for-trading-40441ecdd84d

def calculate_portfolio_volatility(weights, matrix):
    
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_risk_contribution(weights, matrix):
    portfolio_volatility = calculate_portfolio_volatility(weights, matrix)
    marginal_contribution_zaehler = np.dot(matrix, weights)
    risk_contribution = np.multiply(weights, marginal_contribution_zaehler) / portfolio_volatility
    return risk_contribution

def risk_parity_objective(weights, matrix):
    risk_contribution = calculate_risk_contribution(weights, matrix)
    target_risk = np.mean(risk_contribution)
    return np.sum((risk_contribution - target_risk) ** 2)

In [14]:
def get_risk_parity_weights(matrix):
    
    number_of_assets = matrix.shape[0]
    w0 = 1/number_of_assets
    initial_weights = np.full(number_of_assets, w0)
    
    constraints = ({'type': 'eq', 'fun': lambda initial_weights: np.sum(initial_weights) - 1})
    bounds = [[0, 1] for _ in range(number_of_assets)]
    
    result = minimize(risk_parity_objective, initial_weights, args=(matrix), method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return result.x

#### Hierarchical Risk Parity

In [15]:
#Aus dem Paper von Lopez De Prado (2016)
def getIVP(cov,**kargs): # Compute the inverse-variance portfolio 
    ivp=1./np.diag(cov) 
    ivp/=ivp.sum() 
    return ivp

In [16]:
#------------------------------------------------------------------------------ 
def getClusterVar(cov,cItems): # Compute variance per cluster 
    cov_=cov.loc[cItems,cItems] # matrix slice 
    w_=getIVP(cov_).reshape(-1,1) 
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0] 
    return cVar 

In [17]:
#------------------------------------------------------------------------------ 
def getQuasiDiag(link): # Sort clustered items by distance 
    link=link.astype(int) 
    sortIx=pd.Series([link[-1,0],link[-1,1]]) 
    numItems=link[-1,3] # number of original items 
    while sortIx.max()>=numItems: 
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space 
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems 
        sortIx[i]=link[j,0] # item 1 
        df0=pd.Series(link[j,1],index=i+1) 
        sortIx = pd.concat([sortIx, df0]) # item 2  ####Anpassung wegen neuer Pandas-Version!
        sortIx=sortIx.sort_index() # re-sort 
        sortIx.index=range(sortIx.shape[0]) # re-index 
    return sortIx.tolist() 

In [18]:
#------------------------------------------------------------------------------ 
def getRecBipart(cov,sortIx): # Compute HRP allocation
    w = pd.Series(1,index=sortIx) 
    cItems=[sortIx] # initialize all items in one cluster 
    while len(cItems)>0: 
        #Bi-Section der Cluster
        cItems = [i[j:k] 
                  for i in cItems 
                  for j, k in ((0, len(i)//2), (len(i)//2, len(i))) 
                  if len(i) > 1]
        for i in range(0, len(cItems), 2): # parse in pairs 
            cItems0=cItems[i] # cluster 1 
            cItems1=cItems[i+1] # cluster 2 
            cVar0=getClusterVar(cov,cItems0) 
            cVar1=getClusterVar(cov,cItems1) 
            alpha=1-cVar0/(cVar0+cVar1) 
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2 
    return w 

In [19]:
#------------------------------------------------------------------------------ 
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1 
    # This is a proper distance metric 
    dist=((1-corr)/2.)**.5 # distance matrix 
    
    #Adjustierung zur condensed_distance_matrix
    condensed_distance_matrix = squareform(X=dist, checks=False)
    
    return condensed_distance_matrix

In [20]:
def hrp_main(corr, cov, returns): 
    
    #3) cluster 
    dist=correlDist(corr) 
    link=sch.linkage(dist,'single') 
    sortIx=getQuasiDiag(link) 
    sortIx=corr.index[sortIx].tolist() # recover labels 
    df0=corr.loc[sortIx,sortIx] # reorder 
    
    #4) Capital allocation 
    hrp=getRecBipart(cov,sortIx) 
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    hrp_sortiert = hrp.reindex(original_reihenfolge)
    
    array = hrp_sortiert.T.values

    return array

#### Hierarchical Equal Risk Contribution Portfolio

In [21]:
#Es wird auf die Implementierung der Riskfolio-Lib zurückgegriffen
#Code adaptiert von #Code adaptiert von https://medium.com/@orenji.eirl/hierarchical-equal-risk-contribution-with-python-and-riskfolio-lib-ec45dd0f9899

def herc (returns):

    # Building the portfolio object
    port = hc(returns=returns)

    # Estimate optimal portfolio:

    model='HERC'
    correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          correlation=correlation,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

In [22]:
def herc_ledoit (returns):

    #Building the portfolio object
    port = hc(returns=returns)
    
    shrinkage_herc = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix_herc = pd.DataFrame(
        shrinkage_herc.covariance_,
        index = returns.columns,
        columns = returns.columns)

    #Estimate optimal portfolio:

    model='HERC'
    #correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          custom_cov = shrinkage_matrix_herc,
                          codependence='custom_cov',
                          rm=rm,
                          rf=rf,
                          method_cov='custom_cov',   #gemäß https://riskfolio-lib.readthedocs.io/en/latest/hcportfolio.html
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

#### Vergleichs-Metriken

In [23]:
def gesamt_return(gewichte, returns):

    daily_returns = returns @ gewichte
    total = np.prod(1 + daily_returns) - 1
    
    return total

In [24]:
def portfolio_return (gewichte, returns):
    
    avg_returns = []
    avg_returns = returns.mean()
    
    pf_return = sum([gewicht * rendite for gewicht, rendite in zip(gewichte, avg_returns)])
    
    return pf_return

In [25]:
def portfolio_variance (gewichte, returns):
    daily_returns = []
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    pf_var = np.var(daily_returns)
    
    return pf_var

In [26]:
def portfolio_volatility (gewichte, returns):
    variance = portfolio_variance(gewichte, returns)
    volatility = np.sqrt(variance)
    
    return volatility

In [27]:
def annualized_return(gewichte, returns):
    daily_returns = returns @ gewichte
    annual = (np.prod(1 + daily_returns))**(252/daily_returns.shape[0]) - 1
    
    return annual

In [28]:
def annualized_portfolio_return(gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    
    return pf_return*252

In [29]:
def annualized_portfolio_volatility(gewichte, returns):
    pf_volatility = portfolio_volatility(gewichte, returns)
    
    return pf_volatility*np.sqrt(252)

In [30]:
def sharpe_ratio (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [31]:
def sharpe_ratio_one (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [32]:
def sharpe_ratio_two (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [33]:
def annualized_sharpe_ratio (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [34]:
def annualized_sharpe_ratio_one (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [35]:
def annualized_sharpe_ratio_two (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [36]:
def sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [37]:
def sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [38]:
def sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [39]:
def annualized_sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [40]:
def annualized_sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [41]:
def annualized_sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [42]:
def maximum_drawdown (gewichte, returns):
    daily_returns = returns @ gewichte
    returns_kumuliert = (1 + daily_returns).cumprod()
    hochpunkte = returns_kumuliert.cummax()
    
    drawdowns = (returns_kumuliert - hochpunkte) / hochpunkte
    max_drawdown = drawdowns.min()
    
    #print(drawdowns)
    return max_drawdown

In [43]:
def calmar_ratio (gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    max_drawdown = maximum_drawdown(gewichte, returns)
    
    calmar_ratio = pf_return / (-max_drawdown)
    
    return calmar_ratio

In [44]:
def value_at_risk_func(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = np.percentile(sorted_daily_returns, 100*0.05)
    return value_at_risk

In [45]:
#https://github.com/malctaylor15/VaR-and-Expected-Shortfall-Python/blob/master/Expected%20Shortfall%20and%20Value%20at%20Risk.ipynb

def expected_shortfall(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = value_at_risk_func(gewichte, returns)
    
    expected_shortfall = daily_returns[daily_returns <= value_at_risk].mean()
    
    return expected_shortfall

#### Sammel-Funktionen zum Berechnen und Speichern der Metriken

In [46]:
def metriken_equally_weighted(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile1 = {"Methode": "Equally-Weighted",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile1[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile1])], ignore_index=True)

In [47]:
def metriken_minimum_variance(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile2 = {"Methode": "Minimum_Variance",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile2[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile2])], ignore_index=True)

In [48]:
def metriken_max_diversification(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile3 = {"Methode": "Max-Diversification",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile3[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile3])], ignore_index=True)

In [49]:
def metriken_inverse_volatility(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile4 = {"Methode": "Inverse-Volatility",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile4[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile4])], ignore_index=True)

In [50]:
def metriken_risk_parity(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile5 = {"Methode": "Risk-Parity",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile5[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile5])], ignore_index=True)

In [51]:
def metriken_hrp(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile6 = {"Methode": "HRP",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile6[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile6])], ignore_index=True)

In [52]:
def metriken_herc(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile7 = {"Methode": "HERC",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile7[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile7])], ignore_index=True)

## Erzeugung Ergebnisse S&P500

In [53]:
#Tägliche Renditen für den gesamten Zeitraum von 20 Jahren SP500

returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [54]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 3
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    gewichte_equally_weighted = equally_weighted(training_daten)
    gewichte_min_variance= min_variance_opt(cov_matrix)
    gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": gewichte_equally_weighted,
    "Minimum-Variance": gewichte_min_variance,
    "Maximum-Diversification": gewichte_max_diversification,
    "Inverse-Volatility": gewichte_inverse_volatility,
    "Equal-Risk-Contribution": gewichte_risk_parity,
    "HRP": gewichte_hrp,
    "HERC": gewichte_herc
}
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
     #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df = pd.concat(portfolio_returns_list)
portfolio_returns_df = portfolio_returns_df.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.560718656371804e-05
            Iterations: 299
            Function evaluations: 78936
            Gradient evaluations: 299
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0661056745424897
            Iterations: 87
            Function evaluations: 22007
            Gradient evaluations: 83
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6806677955014559e-09
            Iterations: 21
            Function evaluations: 5544
            Gradient evaluations: 21
2005-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6665595056604733e-05
            Iterations: 314
            Function evaluations: 84466
            Gradient evaluations: 314
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8007067307721083
            Iterations: 29
            Function evaluations: 8514
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.060251469829356e-10
            Iterations: 110
            Function evaluations: 32230
            Gradient evaluations: 110
2008-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00021785950399401069
            Iterations: 103
            Function evaluations: 30591
            Gradient evaluations: 103
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.7961748327521527
            Iterations: 28
            Function evaluations: 8333
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3683145155027784e-10
    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1344104499274078e-09
            Iterations: 54
            Function evaluations: 17659
            Gradient evaluations: 54
2011-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.003279946988477e-05
            Iterations: 247
            Function evaluations: 81263
            Gradient evaluations: 247
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2148984941533705
            Iterations: 41
            Function evaluations: 12201
            Gradient evaluations: 37
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0174910689044592e-09
            Iterations: 69
            Function evaluations: 22701
            Gradient evaluations: 69
2011-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.94

2014-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.8497664736703935e-05
            Iterations: 295
            Function evaluations: 106200
            Gradient evaluations: 295
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.519850585120738
            Iterations: 39
            Function evaluations: 14077
            Gradient evaluations: 39
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2051056722236238e-09
            Iterations: 15
            Function evaluations: 5400
            Gradient evaluations: 15
2014-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.652785653865941e-05
            Iterations: 334
            Function evaluations: 120575
            Gradient evaluations: 334
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -3.243407367905203
            Iterations: 70
            Function evaluations: 26856
            Gradient evaluations: 66
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8409430955294662e-09
            Iterations: 19
            Function evaluations: 7715
            Gradient evaluations: 19
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.8323271810440742e-05
            Iterations: 394
            Function evaluations: 161146
            Gradient evaluations: 394
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1327031448446028
            Iterations: 52
            Function evaluations: 19676
            Gradient evaluations: 48
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.555382439330244

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2116690237596855e-09
            Iterations: 50
            Function evaluations: 21900
            Gradient evaluations: 50
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2570472725114772e-05
            Iterations: 335
            Function evaluations: 147065
            Gradient evaluations: 335
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.460081569409615
            Iterations: 79
            Function evaluations: 33011
            Gradient evaluations: 75
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.4301739724156193e-09
            Iterations: 28
            Function evaluations: 12292
            Gradient evaluations: 28
2021-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4294372767715442e-05
            Iterations: 321
            Function evaluations: 149908
            Gradient evaluations: 321
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -4.357105474789429
            Iterations: 94
            Function evaluations: 42146
            Gradient evaluations: 90
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.940671184612122e-09
            Iterations: 33
            Function evaluations: 15411
            Gradient evaluations: 33
2024-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4741685893009108e-05
            Iterations: 358
            Function evaluations: 167187
            Gradient evaluations: 358
Positive directional derivative for linesearch    (Exit mode 8)
            Curre

In [55]:
#Exportieren nach CSV
portfolio_returns_df.to_csv("renditensp500_3Monate_f.csv")

## Ergebnis-Erzeugung JSE

In [56]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [57]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 3
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    JSE_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": JSE_gewichte_equally_weighted,
    "Minimum-Variance": JSE_gewichte_min_variance,
    "Maximum-Diversification": JSE_gewichte_max_diversification,
    "Inverse-Volatility": JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": JSE_gewichte_risk_parity,
    "HRP": JSE_gewichte_hrp,
    "HERC": JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
     #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_JSE = portfolio_returns_df_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3134440810290774e-05
            Iterations: 152
            Function evaluations: 10184
            Gradient evaluations: 152
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.653137425667317
            Iterations: 53
            Function evaluations: 3590
            Gradient evaluations: 53
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1202687138208137e-09
            Iterations: 44
            Function evaluations: 2948
            Gradient evaluations: 44
2005-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.2087758665013425e-05
            Iterations: 122
            Function evaluations: 8053
            Gradient evaluations: 122
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.23

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0787953783759843
            Iterations: 47
            Function evaluations: 3607
            Gradient evaluations: 47
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.131393225491523e-10
            Iterations: 111
            Function evaluations: 8436
            Gradient evaluations: 111
2008-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.876018550733002e-05
            Iterations: 153
            Function evaluations: 11628
            Gradient evaluations: 153
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.989239863956551
            Iterations: 41
            Function evaluations: 3152
            Gradient evaluations: 41
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.162066781630115e-10
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.517710198813543e-09
            Iterations: 45
            Function evaluations: 4096
            Gradient evaluations: 45
2011-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.016842576135813e-05
            Iterations: 204
            Function evaluations: 18768
            Gradient evaluations: 204
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9652678972243303
            Iterations: 61
            Function evaluations: 5584
            Gradient evaluations: 60
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.312570565569058e-09
            Iterations: 46
            Function evaluations: 4232
            Gradient evaluations: 46
2011-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.5750343

2014-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.731418793408144e-05
            Iterations: 220
            Function evaluations: 24421
            Gradient evaluations: 220
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.611761564708866
            Iterations: 73
            Function evaluations: 7736
            Gradient evaluations: 69
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.7458579156896695e-09
            Iterations: 41
            Function evaluations: 4551
            Gradient evaluations: 41
2014-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9016949360606556e-05
            Iterations: 229
            Function evaluations: 26106
            Gradient evaluations: 229
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.52

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.627634522652788
            Iterations: 94
            Function evaluations: 11792
            Gradient evaluations: 90
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.793813600985576e-09
            Iterations: 35
            Function evaluations: 4516
            Gradient evaluations: 35
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6313420519127e-05
            Iterations: 258
            Function evaluations: 34830
            Gradient evaluations: 258
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.854387669049658
            Iterations: 175
            Function evaluations: 23959
            Gradient evaluations: 171
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3810356888071356e-09
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.843814025626564e-09
            Iterations: 61
            Function evaluations: 8968
            Gradient evaluations: 61
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3600606143243473e-05
            Iterations: 242
            Function evaluations: 36784
            Gradient evaluations: 242
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6.432993241014148
            Iterations: 115
            Function evaluations: 17081
            Gradient evaluations: 111
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5836595107466267e-09
            Iterations: 63
            Function evaluations: 9576
            Gradient evaluations: 63
2021-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.466

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.090650657560652e-05
            Iterations: 274
            Function evaluations: 41648
            Gradient evaluations: 274
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -6.42072094048574
            Iterations: 124
            Function evaluations: 18525
            Gradient evaluations: 120
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.288669387680618e-09
            Iterations: 49
            Function evaluations: 7448
            Gradient evaluations: 49
2024-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0351648265315334e-05
            Iterations: 262
            Function evaluations: 39824
            Gradient evaluations: 262
Optimization terminated successfully    (Exit mode 0)
            Current function v

In [58]:
#Exportieren nach CSV
portfolio_returns_df_JSE.to_csv("returnsJSE_3Monate_f.csv")

## Ledoit-Wolf Shrinkage

In [59]:
#Definition von Ledoit-Shrinkage

def ledoit_shrink (returns):
    
    shrinkage = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix = pd.DataFrame(
        shrinkage.covariance_,
        index = returns.columns,
        columns = returns.columns)
    
    return shrinkage_matrix

### Auswertung S&P500 mit Ledoit Wolf

In [60]:
returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [61]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 3
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_gewichte_min_variance,
    "Maximum-Diversification": shrinked_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_gewichte_risk_parity,
    "HRP": shrinked_gewichte_hrp,
    "HERC": shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
     #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked = portfolio_returns_df_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5288862200654737e-05
            Iterations: 308
            Function evaluations: 81312
            Gradient evaluations: 308
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1767340633144645
            Iterations: 80
            Function evaluations: 20138
            Gradient evaluations: 76
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.367469318835228e-09
            Iterations: 9
            Function evaluations: 2376
            Gradient evaluations: 9
2005-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6519176079888082e-05
            Iterations: 259
            Function evaluations: 69672
            Gradient evaluations: 259
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.02

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8382220777257294
            Iterations: 27
            Function evaluations: 7931
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0108461465443225e-10
            Iterations: 110
            Function evaluations: 32231
            Gradient evaluations: 110
2008-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00021805138601100731
            Iterations: 104
            Function evaluations: 30889
            Gradient evaluations: 104
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.837479755165867
            Iterations: 27
            Function evaluations: 8035
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3112231541240667e-10
    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0960326692756335e-09
            Iterations: 55
            Function evaluations: 17986
            Gradient evaluations: 55
2011-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.0355934422769565e-05
            Iterations: 230
            Function evaluations: 75670
            Gradient evaluations: 230
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2540966887926555
            Iterations: 43
            Function evaluations: 12864
            Gradient evaluations: 39
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.83960097191843e-10
            Iterations: 69
            Function evaluations: 22701
            Gradient evaluations: 69
2011-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.933

2014-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.744460783183104e-05
            Iterations: 281
            Function evaluations: 101160
            Gradient evaluations: 281
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.545225387864155
            Iterations: 50
            Function evaluations: 18050
            Gradient evaluations: 50
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.155189270604822e-09
            Iterations: 15
            Function evaluations: 5400
            Gradient evaluations: 15
2014-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.530322012727257e-05
            Iterations: 270
            Function evaluations: 97470
            Gradient evaluations: 270
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.48

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.319475362111676
            Iterations: 78
            Function evaluations: 30117
            Gradient evaluations: 74
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.053140322402451e-09
            Iterations: 9
            Function evaluations: 3655
            Gradient evaluations: 9
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.726437042699659e-05
            Iterations: 310
            Function evaluations: 126790
            Gradient evaluations: 310
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.182355268261843
            Iterations: 57
            Function evaluations: 21724
            Gradient evaluations: 53
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.3933042177635975e-09
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1836628441719586e-09
            Iterations: 38
            Function evaluations: 16645
            Gradient evaluations: 38
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1927390158292476e-05
            Iterations: 332
            Function evaluations: 145749
            Gradient evaluations: 332
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.5151323886742833
            Iterations: 92
            Function evaluations: 38738
            Gradient evaluations: 88
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.223571108605845e-09
            Iterations: 28
            Function evaluations: 12292
            Gradient evaluations: 28
2021-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3830694044519194e-05
            Iterations: 306
            Function evaluations: 142902
            Gradient evaluations: 306
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -4.4353991787064535
            Iterations: 100
            Function evaluations: 44944
            Gradient evaluations: 96
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6403127585131448e-09
            Iterations: 20
            Function evaluations: 9340
            Gradient evaluations: 20
2024-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.428331056071716e-05
            Iterations: 267
            Function evaluations: 124689
            Gradient evaluations: 267
Positive directional derivative for linesearch    (Exit mode 8)
            Curr

In [62]:
#Exportieren nach CSV
portfolio_returns_df_shrinked.to_csv("returnssp500shrinked_3Monate_f.csv")

### Auswertung JSE mit Ledoit Wolf

In [63]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [64]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 3
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_JSE_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_JSE_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_JSE_gewichte_min_variance,
    "Maximum-Diversification": shrinked_JSE_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_JSE_gewichte_risk_parity,
    "HRP": shrinked_JSE_gewichte_hrp,
    "HERC": shrinked_JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked_JSE = portfolio_returns_df_shrinked_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.240047133151458e-05
            Iterations: 135
            Function evaluations: 9045
            Gradient evaluations: 135
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.741709713501023
            Iterations: 43
            Function evaluations: 2923
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.635081070068458e-09
            Iterations: 44
            Function evaluations: 2949
            Gradient evaluations: 44
2005-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.140650188809402e-05
            Iterations: 128
            Function evaluations: 8449
            Gradient evaluations: 128
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.298075

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1478877285111393
            Iterations: 43
            Function evaluations: 3308
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.42068619686613e-10
            Iterations: 102
            Function evaluations: 7753
            Gradient evaluations: 102
2008-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.92960378759943e-05
            Iterations: 133
            Function evaluations: 10108
            Gradient evaluations: 133
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.0724485311548997
            Iterations: 41
            Function evaluations: 3154
            Gradient evaluations: 41
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.224914217875113e-10
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.6990165342171655e-09
            Iterations: 45
            Function evaluations: 4096
            Gradient evaluations: 45
2011-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.031471054283104e-05
            Iterations: 184
            Function evaluations: 16928
            Gradient evaluations: 184
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.087361893195729
            Iterations: 63
            Function evaluations: 5859
            Gradient evaluations: 63
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.128987067954324e-09
            Iterations: 46
            Function evaluations: 4232
            Gradient evaluations: 46
2011-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.6270922

2014-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6676292689534363e-05
            Iterations: 180
            Function evaluations: 19980
            Gradient evaluations: 180
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.773148238486261
            Iterations: 124
            Function evaluations: 13708
            Gradient evaluations: 120
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.893687363371446e-09
            Iterations: 43
            Function evaluations: 4773
            Gradient evaluations: 43
2014-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8566013187950675e-05
            Iterations: 191
            Function evaluations: 21775
            Gradient evaluations: 191
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.770773860884618
            Iterations: 95
            Function evaluations: 12274
            Gradient evaluations: 94
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.697522570557363e-09
            Iterations: 40
            Function evaluations: 5160
            Gradient evaluations: 40
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6109168058748373e-05
            Iterations: 182
            Function evaluations: 24570
            Gradient evaluations: 182
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.998788344708309
            Iterations: 102
            Function evaluations: 13919
            Gradient evaluations: 102
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.678727277228395e-09
      

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.839125326981554e-09
            Iterations: 51
            Function evaluations: 7498
            Gradient evaluations: 51
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4110048482834201e-05
            Iterations: 178
            Function evaluations: 27056
            Gradient evaluations: 178
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6.6503839497106005
            Iterations: 147
            Function evaluations: 22173
            Gradient evaluations: 143
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.5971729097251633e-09
            Iterations: 41
            Function evaluations: 6232
            Gradient evaluations: 41
2021-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.49

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1218483406704243e-05
            Iterations: 182
            Function evaluations: 27664
            Gradient evaluations: 182
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6.546489960492811
            Iterations: 117
            Function evaluations: 17817
            Gradient evaluations: 116
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.036273113821135e-09
            Iterations: 38
            Function evaluations: 5776
            Gradient evaluations: 38
2024-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1006025477233864e-05
            Iterations: 192
            Function evaluations: 29184
            Gradient evaluations: 192
Optimization terminated successfully    (Exit mode 0)
            Current function value: -6

In [65]:
#Exportieren nach CSV
portfolio_returns_df_shrinked_JSE.to_csv("returnsJSEshrinked_3Monate_f.csv")

### Reduzierter S&P500-Bestand

In [66]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ACN       ADI       ADM  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN       NaN       NaN       NaN   
2005-06-03 -0.005374 -0.044955       NaN       NaN       NaN       NaN   
2005-06-06  0.019119 -0.008368       NaN       NaN       NaN       NaN   
2005-06-07 -0.012235 -0.036393       NaN       NaN       NaN       NaN   
2005-06-08  0.009496  0.010399       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.021710 -0.006889 -0.000417   
2025-05-27  0.025154  0.025298  0.012490  0.018897  0.030503  0.014393   
2025-05-28 -0.003415  0.001049 -0.006920  0.001775 -0.005856 -0.005758   
2025-05-29  0.021645 -0.002345  0.006287  0.005507  0.000603  0.000414   
2025-05-30 -0.012006  0.004501  0.005495 -0.002864 -0.008204 -0.002067   

     

In [67]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 3
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_gewichte_min_variance,
    "Maximum-Diversification": reduziert_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_gewichte_risk_parity,
    "HRP": reduziert_gewichte_hrp,
    "HERC": reduziert_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
   #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert = portfolio_returns_df_reduziert.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.105745668140832e-05
            Iterations: 279
            Function evaluations: 28738
            Gradient evaluations: 279
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5646187894305896
            Iterations: 48
            Function evaluations: 4992
            Gradient evaluations: 48
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3432286542379153e-09
            Iterations: 21
            Function evaluations: 2163
            Gradient evaluations: 21
2005-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1384621664736484e-05
            Iterations: 283
            Function evaluations: 29998
            Gradient evaluations: 283
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.613187884690726
            Iterations: 23
            Function evaluations: 2934
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5837761727261757e-10
            Iterations: 106
            Function evaluations: 13462
            Gradient evaluations: 106
2008-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00023929683855949025
            Iterations: 102
            Function evaluations: 12954
            Gradient evaluations: 102
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6256061268392614
            Iterations: 21
            Function evaluations: 2678
            Gradient evaluations: 21
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9477510387760127e-10
    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.284455640759741e-10
            Iterations: 69
            Function evaluations: 9936
            Gradient evaluations: 69
2011-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3326024854814725e-05
            Iterations: 182
            Function evaluations: 26208
            Gradient evaluations: 182
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.985403756274886
            Iterations: 25
            Function evaluations: 3624
            Gradient evaluations: 25
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.405250404735937e-10
            Iterations: 79
            Function evaluations: 11376
            Gradient evaluations: 79
2011-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.072791

2014-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.195036326640111e-05
            Iterations: 253
            Function evaluations: 40227
            Gradient evaluations: 253
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0890213975264187
            Iterations: 30
            Function evaluations: 4800
            Gradient evaluations: 30
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8679385328812764e-09
            Iterations: 20
            Function evaluations: 3180
            Gradient evaluations: 20
2014-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.008895660820242e-05
            Iterations: 238
            Function evaluations: 38081
            Gradient evaluations: 238
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.06

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.807990244878377
            Iterations: 49
            Function evaluations: 9211
            Gradient evaluations: 49
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.6836073475159696e-09
            Iterations: 20
            Function evaluations: 3740
            Gradient evaluations: 20
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1371516057450396e-05
            Iterations: 260
            Function evaluations: 48620
            Gradient evaluations: 260
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.69384151352532
            Iterations: 45
            Function evaluations: 8460
            Gradient evaluations: 45
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1957864928884883e-09
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.304042260909686e-09
            Iterations: 41
            Function evaluations: 8365
            Gradient evaluations: 41
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5646510822079932e-05
            Iterations: 321
            Function evaluations: 65484
            Gradient evaluations: 321
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.185196039759623
            Iterations: 54
            Function evaluations: 11070
            Gradient evaluations: 54
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1915011509012906e-09
            Iterations: 41
            Function evaluations: 8364
            Gradient evaluations: 41
2021-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.97066

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8104246525629658e-05
            Iterations: 305
            Function evaluations: 64356
            Gradient evaluations: 305
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.7842843929033183
            Iterations: 68
            Function evaluations: 13566
            Gradient evaluations: 64
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5028171951972646e-09
            Iterations: 33
            Function evaluations: 6963
            Gradient evaluations: 33
2024-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9527648592660317e-05
            Iterations: 312
            Function evaluations: 67704
            Gradient evaluations: 312
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3

In [68]:
#Exportieren nach CSV
portfolio_returns_df_reduziert.to_csv("returnsreduziert_3Monate_f.csv")

### Reduziert und Ledoit-Shrinkage

In [69]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ACN       ADI       ADM  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN       NaN       NaN       NaN   
2005-06-03 -0.005374 -0.044955       NaN       NaN       NaN       NaN   
2005-06-06  0.019119 -0.008368       NaN       NaN       NaN       NaN   
2005-06-07 -0.012235 -0.036393       NaN       NaN       NaN       NaN   
2005-06-08  0.009496  0.010399       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.021710 -0.006889 -0.000417   
2025-05-27  0.025154  0.025298  0.012490  0.018897  0.030503  0.014393   
2025-05-28 -0.003415  0.001049 -0.006920  0.001775 -0.005856 -0.005758   
2025-05-29  0.021645 -0.002345  0.006287  0.005507  0.000603  0.000414   
2025-05-30 -0.012006  0.004501  0.005495 -0.002864 -0.008204 -0.002067   

     

In [70]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 12
test_fenster = 3
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 63
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_shrinked_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_shrinked_gewichte_min_variance,
    "Maximum-Diversification": reduziert_shrinked_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_shrinked_gewichte_risk_parity,
    "HRP": reduziert_shrinked_gewichte_hrp,
    "HERC": reduziert_shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
     #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert_shrinked = portfolio_returns_df_reduziert_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.068558709922772e-05
            Iterations: 262
            Function evaluations: 26986
            Gradient evaluations: 262
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6511078138392
            Iterations: 48
            Function evaluations: 4991
            Gradient evaluations: 48
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9801819573744985e-09
            Iterations: 20
            Function evaluations: 2060
            Gradient evaluations: 20
2005-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1337054481984163e-05
            Iterations: 243
            Function evaluations: 25758
            Gradient evaluations: 243
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6223

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6489914692526664
            Iterations: 23
            Function evaluations: 2938
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.488965859949403e-10
            Iterations: 98
            Function evaluations: 12447
            Gradient evaluations: 98
2008-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00024156931363108355
            Iterations: 96
            Function evaluations: 12192
            Gradient evaluations: 96
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6642421010636157
            Iterations: 21
            Function evaluations: 2680
            Gradient evaluations: 21
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.819471908937439e-10
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.974445511070134e-10
            Iterations: 69
            Function evaluations: 9936
            Gradient evaluations: 69
2011-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.337664586375212e-05
            Iterations: 188
            Function evaluations: 27072
            Gradient evaluations: 188
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0231737689931677
            Iterations: 26
            Function evaluations: 3769
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.975450638383764e-10
            Iterations: 79
            Function evaluations: 11376
            Gradient evaluations: 79
2011-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.065146

2014-09-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.097402484783674e-05
            Iterations: 254
            Function evaluations: 40386
            Gradient evaluations: 254
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1182368308311212
            Iterations: 29
            Function evaluations: 4641
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7676309922297037e-09
            Iterations: 20
            Function evaluations: 3180
            Gradient evaluations: 20
2014-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.9004713388664054e-05
            Iterations: 255
            Function evaluations: 40800
            Gradient evaluations: 255
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8724348385774006
            Iterations: 52
            Function evaluations: 9773
            Gradient evaluations: 52
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4347093473889332e-09
            Iterations: 20
            Function evaluations: 3740
            Gradient evaluations: 20
2017-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9993509539989655e-05
            Iterations: 259
            Function evaluations: 48433
            Gradient evaluations: 259
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.74688835979106
            Iterations: 43
            Function evaluations: 8086
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.9320034416149537e-09
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.835342638080953e-09
            Iterations: 41
            Function evaluations: 8365
            Gradient evaluations: 41
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4869255945455282e-05
            Iterations: 287
            Function evaluations: 58548
            Gradient evaluations: 287
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.229270202936395
            Iterations: 57
            Function evaluations: 11482
            Gradient evaluations: 56
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.989243785044526e-09
            Iterations: 41
            Function evaluations: 8364
            Gradient evaluations: 41
2021-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.884588

2023-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7142097511704284e-05
            Iterations: 251
            Function evaluations: 52962
            Gradient evaluations: 251
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.8899534090777754
            Iterations: 68
            Function evaluations: 14419
            Gradient evaluations: 68
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.978264312522957e-09
            Iterations: 20
            Function evaluations: 4221
            Gradient evaluations: 20
2024-03-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8463465219869636e-05
            Iterations: 252
            Function evaluations: 54685
            Gradient evaluations: 252
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.

In [71]:
#Exportieren nach CSV
portfolio_returns_df_reduziert_shrinked.to_csv("returnsreduziertshrinked_3Monate_f.csv")